# Riot API Explorer
Exploración básica de las APIs: match-v5, league-v4 y champion-mastery-v4

In [47]:
# Cargar variables de entorno y configuración
from dotenv import load_dotenv
import os
import requests
import json

load_dotenv()
API_KEY = os.getenv('RIOT_API_KEY')
REGION = os.getenv('RIOT_REGION', 'americas')  # americas, europe, asia
PLATFORM = os.getenv('RIOT_PLATFORM', 'la1')  # na1, euw1, kr, etc.

print(f"API Key configurada: {API_KEY[:10]}..." if API_KEY else "⚠️ API Key no encontrada")
print(f"Region: {REGION}")
print(f"Platform: {PLATFORM}")

API Key configurada: RGAPI-2722...
Region: americas
Platform: la1


# Obtener los nombres de campeones desde data dragon API

In [48]:
# Obtener datos de Data Dragon para traducir IDs a nombres
def get_champion_data():
    """
    Obtiene la información de campeones desde Data Dragon
    Retorna un diccionario con {championId: nombre}
    """
    # Obtener la versión más reciente
    versions_url = "https://ddragon.leagueoflegends.com/api/versions.json"
    versions = requests.get(versions_url).json()
    latest_version = versions[0]
    
    # Obtener datos de campeones
    champions_url = f"https://ddragon.leagueoflegends.com/cdn/{latest_version}/data/es_MX/champion.json"
    response = requests.get(champions_url)
    champions_data = response.json()
    
    # Crear diccionario {id: nombre}
    champion_dict = {}
    for champ_key, champ_info in champions_data['data'].items():
        champion_id = int(champ_info['key'])
        champion_dict[champion_id] = champ_info['name']
    
    print(f"Data Dragon cargado (versión {latest_version})")
    print(f"Total de campeones: {len(champion_dict)}")
    
    return champion_dict

# Cargar datos de campeones
champion_names = get_champion_data()

Data Dragon cargado (versión 15.21.1)
Total de campeones: 171


## 2. Account-V1 + Champion-Mastery-V4: Obtener PUUID y Maestría

In [49]:
# 1. Obtener PUUID usando Account-V1 (Riot ID)
def get_puuid_by_riot_id(game_name, tag_line):
    """
    Obtiene el PUUID de un jugador usando su Riot ID (nombre#tag)
    Ejemplo: game_name="PlayerName", tag_line="NA1"
    """
    url = f"https://{REGION}.api.riotgames.com/riot/account/v1/accounts/by-riot-id/{game_name}/{tag_line}"
    headers = {"X-Riot-Token": API_KEY}
    response = requests.get(url, headers=headers)
    return response.json() if response.status_code == 200 else {"error": response.status_code, "message": response.text}

# 2. Obtener maestría de campeones usando el PUUID
def get_champion_mastery(puuid):
    url = f"https://{PLATFORM}.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-puuid/{puuid}"
    headers = {"X-Riot-Token": API_KEY}
    response = requests.get(url, headers=headers)
    return response.json() if response.status_code == 200 else {"error": response.status_code, "message": response.text}

# Ejemplo: Cambia estos valores por un Riot ID real
game_name = "El Jods"  # Nombre del jugador (sin el #)
tag_line = "LAN"            # Tag (lo que viene después del #)

# Obtener PUUID
account_info = get_puuid_by_riot_id(game_name, tag_line)
print("Obtenido")

Obtenido


In [50]:
# Mostrar maestría de campeones CON NOMBRES
if 'puuid' in account_info:
    puuid = account_info['puuid']
    
    # Obtener maestría de campeones
    mastery_data = get_champion_mastery(puuid)
    
    if isinstance(mastery_data, list):
        print(f"=== Top 10 Campeones por Maestría ===\n")
        for i, champ in enumerate(mastery_data[:10], 1):
            champ_id = champ['championId']
            champ_name = champion_names.get(champ_id, f"Unknown (ID: {champ_id})")
            
            print(f"{i}. {champ_name}")
            print(f"   Puntos: {champ['championPoints']:,}")
            print(f"   Nivel: {champ['championLevel']}")
            print()
    else:
        print(f"Error obteniendo maestría: {mastery_data}")
else:
    print(f" Error obteniendo PUUID: {account_info}")

=== Top 10 Campeones por Maestría ===

1. Lillia
   Puntos: 603,997
   Nivel: 58

2. Qiyana
   Puntos: 566,350
   Nivel: 54

3. Irelia
   Puntos: 407,165
   Nivel: 40

4. Aurora
   Puntos: 192,746
   Nivel: 20

5. Zoe
   Puntos: 186,233
   Nivel: 20

6. Fiddlesticks
   Puntos: 126,192
   Nivel: 14

7. Gragas
   Puntos: 114,220
   Nivel: 13

8. Nidalee
   Puntos: 109,100
   Nivel: 13

9. Ahri
   Puntos: 107,916
   Nivel: 12

10. Bel'Veth
   Puntos: 93,817
   Nivel: 11



### Data Dragon: Traducir IDs de campeones a nombres

## 3. Match-V5: Obtener partidas

In [51]:
# Obtener lista de IDs de partidas de un jugador
def get_match_ids(puuid, count=20):
    url = f"https://{REGION}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids"
    headers = {"X-Riot-Token": API_KEY}
    params = {"count": count}
    response = requests.get(url, headers=headers, params=params)
    return response.json() if response.status_code == 200 else response.text

# Ejemplo: usa el PUUID obtenido anteriormente
match_ids = get_match_ids(puuid, count=5)
print(f"IDs de partidas: {match_ids}")

IDs de partidas: ['LA1_1664172869', 'LA1_1663947386', 'LA1_1663676358', 'LA1_1663666511', 'LA1_1663654721']


In [52]:
# Obtener detalles completos de una partida
def get_match_details(match_id):
    url = f"https://{REGION}.api.riotgames.com/lol/match/v5/matches/{match_id}"
    headers = {"X-Riot-Token": API_KEY}
    response = requests.get(url, headers=headers)
    return response.json() if response.status_code == 200 else response.text


if match_ids and len(match_ids) > 0:
    match_data = get_match_details(match_ids[0])
    print(json.dumps(match_data, indent=2)[:2000])  # Primeros 2000 caracteres

{
  "metadata": {
    "dataVersion": "2",
    "matchId": "LA1_1664172869",
    "participants": [
      "UOx0uHK3lKxlFvyRl46st_rpuWfX3ollOeUBNzwINQ5yNN0o6yEcYeb1EalH8-VSfefw7Tow4OHGfQ",
      "gtsjqrtfxwehLFn2EqcY9e1EwwwXD3GURRxMONcqaCuq9KlLhIVx_EPTe0UA_n6UrGJCpyWJd_u15g",
      "pvyJrJfe4crvArrFFdlvc7P9xa5_GLKQfl2KG_lkWarInFrzH4U0RIcAMI22QYNxQO7bLglXuumV2Q",
      "xY9tqeu5Rp4fpi7_8RZeGbpmuwYy2XQHfQ2K8jxC46fmoVKFjnNjl83doWRqhwwj4K7v8oIaMpYF5A",
      "pcUErK7AFrQhcsVwaRsvJQGoNqLZ29_qNoKM6hfyxCDPs1AK5bpGR-5EEwg1XhQBMaSRqsHckx40CA",
      "qZ8vQhXysUEIfsstKAUGftedVCU6w44vqKGFWwT3kgbeN9wV7Ywt-V9-lNWETtPO6mi1CnHkfnF3cA",
      "27DsLD4W8AOFwg4FIuoBaSvIjRPyrYpzAWhOOo573toqkmTlk812yIcX3g7LIRnkPIJx9NaI8BtuWw",
      "uB9TmAQwjru6qe693ZVPKfaYIximTkTc35W_MhzgrZi4vzdAcyHQ2N_O1GDn1tzDIQZhsf9cmfkB7Q",
      "4u-eLtar3BD4hTkx3meS3GsbU8HjEb4DMY8EWLt5w5auzTULXSTy6CANNFbWDg4y38LkxbtVAvX1ZA",
      "aWLUPQZSmWJ5NASeN-6F2KRPkpK4MCtn3y1i5zkvIh3pJOiCxJfK_UP6OKn3Grz0izxR-6igtUW_jw"
    ]
  },
  "info": {
 

In [53]:
# Ver información específica de una partida
# if match_ids and len(match_ids) > 0:
#     match_data = get_match_details(match_ids[0])
#     
#     print("=== Información General ===")
#     print(f"Game Mode: {match_data['info']['gameMode']}")
#     print(f"Game Duration: {match_data['info']['gameDuration']} segundos")
#     print(f"\n=== Participantes ===")
#     
#     for participant in match_data['info']['participants']:
#         print(f"{participant['summonerName']}: {participant['championName']} - {participant['kills']}/{participant['deaths']}/{participant['assists']}")

## Ejemplo completo: Análisis rápido de un jugador

In [54]:
# Flujo completo: análisis de un jugador usando Riot ID
def analyze_summoner(game_name: str, tag_line: str, top_champs: int = 5, match_count: int = 10):
    """
    Analiza un jugador completo: maestría de campeones y últimas partidas.
    
    Args:
        game_name (str): Nombre del jugador (sin el #)
        tag_line (str): Tag del jugador (después del #)
        top_champs (int): Número de campeones top por maestría a mostrar
        match_count (int): Número de partidas recientes a obtener
    """
    print(f"Analizando jugador: {game_name}#{tag_line}\n")
    
    # 1. Obtener PUUID usando Account-V1
    account = get_puuid_by_riot_id(game_name, tag_line)
    if 'puuid' not in account:
        print(f"Error obteniendo cuenta: {account}")
        return None
    
    player_puuid = account['puuid']
    print(f"PUUID obtenido: {player_puuid[:20]}...\n")
    
    # 2. Maestría de campeones (con nombres desde Data Dragon)
    mastery = get_champion_mastery(player_puuid)
    if isinstance(mastery, list) and len(mastery) > 0:
        print(f"Top {top_champs} Campeones por Maestría:")
        for i, champ in enumerate(mastery[:top_champs], 1):
            champ_id = champ['championId']
            champ_name = champion_names.get(champ_id, f"Unknown (ID: {champ_id})")
            print(f"  {i}. {champ_name}")
            print(f"     Puntos: {champ['championPoints']:,} | Nivel: {champ['championLevel']}")
    else:
        print(f"No se pudo obtener maestría: {mastery}")
    
    # 3. Últimas partidas usando Match-V5
    matches = get_match_ids(player_puuid, count=match_count)
    if isinstance(matches, list) and len(matches) > 0:
        print(f"\nÚltimas {len(matches)} partidas:")
        for i, match_id in enumerate(matches[:5], 1):  # Mostrar solo 5 IDs
            print(f"  {i}. {match_id}")
        if len(matches) > 5:
            print(f"  ... y {len(matches) - 5} más")
        
        return {
            'puuid': player_puuid,
            'mastery': mastery[:top_champs],
            'match_ids': matches
        }
    else:
        print(f"\nNo se pudieron obtener partidas: {matches}")
        return None

In [55]:
# Ejemplo de uso (ejecuta esta celda para analizar un jugador)
resultado = analyze_summoner("El Jods", "LAN", top_champs=3, match_count=5)

Analizando jugador: El Jods#LAN

PUUID obtenido: gtsjqrtfxwehLFn2EqcY...

PUUID obtenido: gtsjqrtfxwehLFn2EqcY...

Top 3 Campeones por Maestría:
  1. Lillia
     Puntos: 603,997 | Nivel: 58
  2. Qiyana
     Puntos: 566,350 | Nivel: 54
  3. Irelia
     Puntos: 407,165 | Nivel: 40
Top 3 Campeones por Maestría:
  1. Lillia
     Puntos: 603,997 | Nivel: 58
  2. Qiyana
     Puntos: 566,350 | Nivel: 54
  3. Irelia
     Puntos: 407,165 | Nivel: 40

Últimas 5 partidas:
  1. LA1_1664172869
  2. LA1_1663947386
  3. LA1_1663676358
  4. LA1_1663666511
  5. LA1_1663654721

Últimas 5 partidas:
  1. LA1_1664172869
  2. LA1_1663947386
  3. LA1_1663676358
  4. LA1_1663666511
  5. LA1_1663654721


In [63]:
# Ejemplo de uso: Analizar las últimas 5 partidas del jugador
if 'puuid' in locals() and puuid:
    detailed_stats = analyze_recent_matches(puuid, match_count=15)
else:
    print("Primero ejecuta las celdas anteriores para obtener un PUUID")

Analizando 15 partidas...

Procesando partida 1/15: LA1_1664172869
  Rol: JUNGLE | Campeón: Nidalee | Victoria: Sí | Oro ~15min: 6,738
Procesando partida 2/15: LA1_1663947386
  Rol: JUNGLE | Campeón: Nidalee | Victoria: Sí | Oro ~15min: 6,738
Procesando partida 2/15: LA1_1663947386
  Rol: MIDDLE | Campeón: Azir | Victoria: Sí | Oro ~15min: 6,508
Procesando partida 3/15: LA1_1663676358
  Rol: MIDDLE | Campeón: Azir | Victoria: Sí | Oro ~15min: 6,508
Procesando partida 3/15: LA1_1663676358
  Rol: BOTTOM | Campeón: Xayah | Victoria: Sí | Oro ~15min: 6,886
Procesando partida 4/15: LA1_1663666511
  Rol: BOTTOM | Campeón: Xayah | Victoria: Sí | Oro ~15min: 6,886
Procesando partida 4/15: LA1_1663666511
  Rol: MIDDLE | Campeón: Azir | Victoria: No | Oro ~15min: 5,413
Procesando partida 5/15: LA1_1663654721
  Rol: MIDDLE | Campeón: Azir | Victoria: No | Oro ~15min: 5,413
Procesando partida 5/15: LA1_1663654721
  Rol: JUNGLE | Campeón: Qiyana | Victoria: Sí | Oro ~15min: 6,953
Procesando partida

In [62]:
def analyze_recent_matches(puuid: str, match_count: int = 5):
    """
    Analiza las últimas partidas de un jugador para obtener estadísticas específicas.
    
    Args:
        puuid (str): PUUID del jugador
        match_count (int): Número de partidas a analizar
        
    Returns:
        dict: Diccionario con estadísticas agregadas por rol
    """
    # Obtener IDs de partidas
    match_ids = get_match_ids(puuid, count=match_count)
    
    if not isinstance(match_ids, list) or len(match_ids) == 0:
        print(f"No se pudieron obtener partidas para el PUUID: {puuid}")
        return None
    
    print(f"Analizando {len(match_ids)} partidas...\n")
    
    # Estructura para almacenar datos por rol
    stats_by_role = {}
    overall_stats = {
        'total_games': 0,
        'wins': 0,
        'champions_played': [],
        'bans': []
    }
    
    for idx, match_id in enumerate(match_ids, 1):
        print(f"Procesando partida {idx}/{len(match_ids)}: {match_id}")
        
        match_data = get_match_details(match_id)
        
        if not isinstance(match_data, dict) or 'info' not in match_data:
            print(f"  Error obteniendo datos de la partida")
            continue
        
        # Buscar al jugador en los participantes
        player_data = None
        for participant in match_data['info']['participants']:
            if participant['puuid'] == puuid:
                player_data = participant
                break
        
        if not player_data:
            print(f"  Jugador no encontrado en esta partida")
            continue
        
        # Extraer datos relevantes
        role = player_data.get('teamPosition', 'UNKNOWN')
        champion_id = player_data['championId']
        champion_name = champion_names.get(champion_id, f"ID:{champion_id}")
        won = player_data['win']
        
        # Oro a los 15 minutos (timeline puede no estar disponible en todas las partidas)
        # Usamos goldEarned total como aproximación si no hay timeline
        gold_at_15 = player_data.get('challenges', {}).get('goldPerMinute', 0) * 15
        if gold_at_15 == 0:
            # Fallback: estimar basado en oro total y duración
            total_gold = player_data.get('goldEarned', 0)
            game_duration_minutes = match_data['info']['gameDuration'] / 60
            if game_duration_minutes > 0:
                gold_per_min = total_gold / game_duration_minutes
                gold_at_15 = gold_per_min * min(15, game_duration_minutes)
        
        # Ban del jugador (puede ser None si no baneó)
        # Los bans están en el nivel de equipo
        team_id = player_data['teamId']
        player_ban = None
        for team in match_data['info']['teams']:
            if team['teamId'] == team_id:
                # Encontrar el ban específico del jugador (no siempre está identificado por jugador)
                # Usamos el primer ban del equipo como aproximación
                if 'bans' in team and len(team['bans']) > 0:
                    ban_champion_id = team['bans'][0].get('championId', -1)
                    if ban_champion_id != -1:
                        player_ban = champion_names.get(ban_champion_id, f"ID:{ban_champion_id}")
                break
        
        # Agregar a estadísticas por rol
        if role not in stats_by_role:
            stats_by_role[role] = {
                'games': 0,
                'wins': 0,
                'total_gold_15min': 0,
                'champions': []
            }
        
        stats_by_role[role]['games'] += 1
        stats_by_role[role]['wins'] += 1 if won else 0
        stats_by_role[role]['total_gold_15min'] += gold_at_15
        stats_by_role[role]['champions'].append(champion_name)
        stats_by_role[role]['champions'].count(champion_name)
        
        # Estadísticas generales
        overall_stats['total_games'] += 1
        overall_stats['wins'] += 1 if won else 0
        overall_stats['champions_played'].append(champion_name)
        if player_ban:
            overall_stats['bans'].append(player_ban)
        
        print(f"  Rol: {role} | Campeón: {champion_name} | Victoria: {'Sí' if won else 'No'} | Oro ~15min: {gold_at_15:,.0f}")
    
    # Calcular promedios
    print(f"\n{'='*60}")
    print(f"RESUMEN DE {overall_stats['total_games']} PARTIDAS")
    print(f"{'='*60}\n")
    
    print(f"Victorias: {overall_stats['wins']}/{overall_stats['total_games']} ({overall_stats['wins']/overall_stats['total_games']*100:.1f}%)")
    print(f"\nCampeones jugados: {', '.join(overall_stats['champions_played'])}")
    print(f"Baneos realizados: {', '.join(overall_stats['bans']) if overall_stats['bans'] else 'N/A'}")
    
    print(f"\n{'='*60}")
    print(f"ESTADÍSTICAS POR ROL")
    print(f"{'='*60}\n")
    
    for role, stats in stats_by_role.items():
        avg_gold = stats['total_gold_15min'] / stats['games']
        win_rate = (stats['wins'] / stats['games']) * 100
        print(f"{role}:")
        print(f"  Partidas: {stats['games']}")
        print(f"  Win Rate: {stats['wins']}/{stats['games']} ({win_rate:.1f}%)")
        print(f"  Oro promedio a los 15min: {avg_gold:,.0f}")
        print(f"  Campeones: {', '.join(stats['champions'])}")
        print(f" Contador de campeones jugados: { {champ: stats['champions'].count(champ) for champ in set(stats['champions'])} }")
        print()
    
    return {
        'overall': overall_stats,
        'by_role': stats_by_role
    }

## Análisis detallado de partidas: Oro, campeones, victorias y baneos